In [ ]:
%pip install -q streamlit tensorflow pillow firebase-admin numpy

In [1]:
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image

# -----------------------------
# Cargar modelo entrenado
# -----------------------------
# Asegúrate de haber guardado tu modelo con:
# model.save("mlp_mnist.h5")
model = tf.keras.models.load_model("mlp_mnist.h5")

st.title("🔢 Clasificador de Números MNIST")
st.write("Sube una imagen de un número escrito a mano (0-9) y la IA intentará reconocerlo.")

# -----------------------------
# Subida de imagen
# -----------------------------
uploaded_file = st.file_uploader("📤 Sube una imagen", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Mostrar la imagen original
    image = Image.open(uploaded_file).convert("L")  # Escala de grises
    st.image(image, caption="Imagen subida", width=150)

    # -----------------------------
    # Preprocesamiento
    # -----------------------------
    img_resized = image.resize((28, 28))  # Redimensionar a 28x28
    img_array = np.array(img_resized)

    # Invertir colores si está en blanco sobre negro
    if np.mean(img_array) > 127:
        img_array = 255 - img_array

    img_array = img_array / 255.0          # Normalizar a [0,1]
    img_array = img_array.reshape(1, 28, 28)

    # -----------------------------
    # Predicción
    # -----------------------------
    pred = model.predict(img_array)
    predicted_digit = np.argmax(pred)

    st.subheader(f"✅ Predicción: **{predicted_digit}**")


2025-09-30 09:17:34.769645: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-30 09:17:35.131112: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-30 09:17:53.057066: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-30 09:18:05.950137: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

KeyboardInterrupt: 

In [ ]:
!streamlit run app.ipynb --server.headless true --server.port 8501